In [ ]:
import os
import sys
# Get the parent directory of the current working directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to sys.path
sys.path.append(parent_dir)



In [14]:
import gdelt # for gdelt searchs
from tools.gkg_tools import * # for gkg searchs
# %run "../gkg_tools.py" # using magic command run to access the script from the parent directory

from nb_widgets import * # for widgets

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset


import nltk
from nltk.tokenize import word_tokenize, sent_tokenize


from transformers import pipeline, set_seed
import torch

# # GPU Timing (using GPU 1) else -1 for CPU
device_id = 1 if torch.cuda.is_available() else -1 

## Get GKG Data

In [ ]:
field_options = ['amounts', 'persons', 'v2persons', 'allnames', 'themes', 'v2tone']
dropdown = create_dropdown_widget(field_options)

Dropdown(description='Select:', options=('amounts', 'persons', 'v2persons', 'allnames', 'themes', 'v2tone'), v…

Selected option: allnames
Selected option: persons
Selected option: themes
Selected option: allnames


In [18]:
gkg = gkg_operator() # create a gkg operator
manga = pd.read_csv('..\\data\\manga_soup.csv') # We could alternatively use the subset of the queried data in the last section.
manga = manga[manga['sourcecommonname'].map(manga['sourcecommonname'].value_counts()) > 5]
gkg.get_gkg(data=manga)
gkg.parse_urls() # parse urls in the gkg_query
gkg.parse_gkg_field(dropdown.value)




,index,allnames_0,allnames_1
0,0,Sun God Nika,47
1,0,Sun God,133
2,0,Elbaf Arc,351
3,0,One Piece,700
4,0,Sun God,918
...,...,...,...
5423,257,One Piece,7244
5424,257,Shonen Jump,7394
5425,257,Manga Plus,7413
5426,257,One Piece,7471


In [3]:

# gkg.vectorize_field() # default weighting is 'uniform'
gkg.vectorize_field(weight='weighted') # logaritmic weighting
# gkg.vectorized_df.columns = gkg.tokenize_field(col_idx=2) # resets columns to uncleaned field names
gkg.vectorized_df.head(2)
# print(gkg.vectorized_df.shape)

/home/cld1465/notebooks/gdelt/gkg_tools.py:167: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  aligned_tokens = aligned_tokens.applymap(get_weight)  # Apply weighting function to each token count


,aaron dismuke fullmetal,abby trott,adam gibbs,adam mcarthur,aida iro,aiki hirose,aira shiratori,aizawa shota,akane kurokawa,akari kito,...,yuyuko takemiya,yuyuko takemiya yasu,yuzuru shimazaki ginga tetsudo,zach aguilar,zaraki kenpachi,zenitsu agatsuma,zeno robinson,zhang chulan,zhenhua gu,zyuden sentai kyoryuger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
gkg.get_fields_stats(weight='weighted')

Top 10 Tokens by Non-Zero Percentage
                Non-Zero Percentage
eiichiro oda              10.465116
akira toriyama             7.364341
saka mikami                5.426357
cocoa fujiwara             5.038760
ayane sakura               4.651163
kaito ishikawa             4.263566
kazuya nakai               3.875969
izuku midoriya             3.488372
natsuki hanae              3.100775
nana mizuki                3.100775

Top 10 Tokens by Mean Weighted Value
                      Mean Weighted Value
zeno robinson                    3.806662
yuji kaku                        3.135494
mateus manhanini                 3.135494
yuji kaku creees lee             2.564949
scott snyder                     2.302585
nick dragotta                    2.197225
azami kurotani                   2.197225
trina nishimura                  2.063567
shojo manga                      1.945910
kamisama kis                     1.791759


## Zero-shot classification

In [26]:

# Initialize the zero-shot classification pipeline
pipe = pipeline("zero-shot-classification",
                model="facebook/bart-large-mnli",
                device=device_id)
labels = ['one piece', 'dan da dan', 'dragon ball',
          'star wars', 'naruto', 'hero academia',
          'fruits basket', 'fairy tale', 'attack on titan']
titles = gkg.gkg_query['title'].tolist()

# Create a Hugging Face Dataset
data = {"title": titles}
dataset = Dataset.from_dict(data)

# Define a function to classify the titles
def classify(example):
    result = pipe(example['title'], candidate_labels=labels)
    return {
        'labels': result['labels'],
        'scores': result['scores']
    }

# Apply classification to the entire dataset in a batched manner
results = dataset.map(classify, batched=False)  # Use batched=True for faster processing

# Convert results to a DataFrame
df_results = pd.DataFrame(columns=labels)

# Populate the DataFrame with the classification scores
for idx, row in enumerate(results):
    label_scores = dict(zip(row['labels'], row['scores']))
    df_results.loc[idx] = [label_scores.get(label, 0) for label in labels]

# Set the 'title' as the index of the DataFrame
df_results['title'] = titles  # Add titles to the DataFrame
df_results.set_index('title', inplace=True)  # Set titles as the index

# Now df_results contains the classification scores for each title, with titles as the index
# df_results


Map:   0%|          | 0/258 [00:00<?, ? examples/s]

In [ ]:

# Define your titles and candidate labels
titles = gkg.gkg_query['title'].tolist()
# labels = ['one piece', 'dan da dan', 'dragon ball', 'star wars', 'naruto', 'hero academia',
#           'fruits basket', 'fairy tale', 'attack on titan']
# labels = ['one piece','manga', 'commentary', 'interview','news','podcast','forecast']
# labels = ['tv', 'movie', 'radio', 'interview', 'news', 'episodes', 'game', 'anime', 'manga', 'actors', 'podcast']
# Create a Hugging Face Dataset
data = {"title": titles}
dataset = Dataset.from_dict(data)

# Define a function to classify the titles
def classify(example):
    result = pipe(example['title'], candidate_labels=labels)
    return {
        'labels': result['labels'],
        'scores': result['scores']
    }

# Apply classification to the entire dataset in a batched manner
results = dataset.map(classify, batched=False)  # Use batched=True for faster processing

# Convert results to a DataFrame
df_results = pd.DataFrame(columns=labels)

# Populate the DataFrame with the classification scores
for idx, row in enumerate(results):
    label_scores = dict(zip(row['labels'], row['scores']))
    df_results.loc[idx] = [label_scores.get(label, 0) for label in labels]

# Set the 'title' as the index of the DataFrame
df_results['title'] = titles  # Add titles to the DataFrame
df_results.set_index('title', inplace=True)  # Set titles as the index

# Now df_results contains the classification scores for each title, with titles as the index
# df_results


Map:   0%|          | 0/258 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [7]:
# Create a new 'label' column based on the condition
def assign_label(row):
    # Get the column name with the maximum value greater than 0.50
    max_value_col = row[row > 0.50].idxmax() if any(row > 0.50) else "NO MATCH"
    return max_value_col

# Apply the function across all rows to create the new 'label' column
df_results['label'] = df_results.apply(assign_label, axis=1)

# Convert the labels to uppercase
df_results['label'] = df_results['label'].str.upper()



In [24]:
df_results[['one piece', 'label']].sort_values(by='one piece',ascending=False)

,one piece,label
title,,
"One Piece: Sun God Loki Vs Sun God Nika, Explained",0.979288,ONE PIECE
One Piece: Oda Reveals A Legendary God From Elbaf,0.978500,ONE PIECE
One Piece Anime To Go On Break Till 2025,0.978488,ONE PIECE
Pirates Who Are Animals In One Piece:,0.973659,ONE PIECE
"One Piece: Why Luffy Will Destroy The World, Explained",0.973164,ONE PIECE
...,...,...
Dan Da Dan: Okarun’s Turbo Granny Powers Explained,0.011448,DAN DA DAN
Akira Toriyama's Swan Song Dragon Ball Daima Has Arrived,0.010300,DRAGON BALL
Dan Da Dan Officially Confirms It's Jujutsu Kaisen's True Successor By Revisiting The Hit Shonen's Big Theme,0.009993,DAN DA DAN


In [9]:
# Reset the index for df_results only
df_results_reset = df_results.reset_index(drop=True)

# Merge based on the row index
merged_manga = pd.concat([manga, df_results_reset], axis=1)

# Display the resulting DataFrame to confirm the merge
merged_manga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  258 non-null    int64  
 1   gkgrecordid                 258 non-null    object 
 2   date                        258 non-null    int64  
 3   sourcecollectionidentifier  258 non-null    int64  
 4   sourcecommonname            258 non-null    object 
 5   documentidentifier          258 non-null    object 
 6   counts                      17 non-null     object 
 7   v2counts                    17 non-null     object 
 8   themes                      258 non-null    object 
 9   v2themes                    258 non-null    object 
 10  locations                   129 non-null    object 
 11  v2locations                 129 non-null    object 
 12  persons                     242 non-null    object 
 13  v2persons                   242 non

### Zero-shot classifer example

In [10]:
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device_id)

# Define your headlines and candidate labels
headlines = [
    "New smartphone model released with advanced AI features",
    "Local team wins championship in overtime thriller",
    "Government announces new economic policy reforms"
]

# Define possible categories
candidate_labels = ["Technology", "Sports", "Politics"]

# Classify each headline
results = [classifier(headline, candidate_labels) for headline in headlines]

# Print the classification results
for i, result in enumerate(results):
    print(f"Headline: {headlines[i]}")
    print(f"Label: {result['labels'][0]} (Score: {result['scores'][0]:.2f})")
    print("-" * 30)

Headline: New smartphone model released with advanced AI features
Label: Technology (Score: 0.95)
------------------------------
Headline: Local team wins championship in overtime thriller
Label: Sports (Score: 0.97)
------------------------------
Headline: Government announces new economic policy reforms
Label: Politics (Score: 0.88)
------------------------------
